<a href="https://colab.research.google.com/github/GuilhermePelegrina/Mackenzie/blob/main/Aulas/2s2024/TIC/Aula_09_Regressao_nao_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/logo_mackenzie.png'>


# **Regressão não linear**

Vimos na aula passada a regressão linear como um modelo de aprendizado de máquina aplicado em problemas de regressão. Um dos pressupostos era que a relação entre variáveis de entrada e variável de saída fosse linear. Ou seja, para um bom ajuste de modelo, essas variáveis deveriam estar relacionadas de uma forma linear (o umais próxima de linear possível).

No entanto, em muitas aplicações as variáveis se relacionam de formas não lineares (quadráticas, exponenciais, etc.). Sendo assim, modelos que levam em conta a não linearidade dos dados tendem a levar a resultados melhores.

Nesta aula, veremos modelos não-lineares de regressão. São várias as não linearidades que podemos modelar a partir do uso de funções polinomiais, exponenciais, logarítmicas ou até trigonométricas. A escolha da função depende dos dados sob análise. E, em geral, o objetivo na etapa de ajuste do modelo é o mesmo da regressão linear. Ou seja, procuramos minimizar a soma dos erros quadráticos entre os valores previstos e os valores reais.


## Exemplo: Auto MPG

Nesta aula, usaremos novamente o conjunto de dados Auto MPG. Esse conjunto de dados pode ser extraído diretamente da biblioteca *seaborn*.

Relembrando, este conjunto de dados descreve o consumo de diferentes carros (medido pelo número de milhas percorridas com o uso de um galão de gasolina) com base no seguinte conjunto de características de tais carros:

- cylinders -> Número de cilíndros.

- displacement -> Capacidade do motor.

- horsepower -> Potência (cavalo-vapor).

- weight -> Peso.

- acceleration -> Tempo, em segundos, até atingir a velocidade de 100 km/h (partindo do carro em repouso).

- model_year -> Ano do modelo do carro

- origin -> Local de produção do carro.

- name -> Nome do modelo do carro.

In [ ]:
# Leitura dos dados

import seaborn as sns
df = sns.load_dataset('mpg')
df.dropna(inplace=True)
df.head()

## Exemplo Auto MPG

Vamos agora aplicar a Regressão Linear Múltipla em um novo conjunto de dados, chamado Auto MPG. Esse conjunto de dados pode ser extraído diretamente da biblioteca *seaborn*.

Este conjunto de dados descreve o consumo de diferentes carros (medido pelo número de milhas percorridas com o uso de um galão de gasolina) com base no seguinte conjunto de características de tais carros:

- cylinders -> Número de cilíndros.

- displacement -> Capacidade do motor.

- horsepower -> Potência (cavalo-vapor).

- weight -> Peso.

- acceleration -> Tempo, em segundos, até atingir a velocidade de 100 km/h (partindo do carro em repouso).

- model_year -> Ano do modelo do carro

- origin -> Local de produção do carro.

- name -> Nome do modelo do carro.

Uma característica interessante nesse dado é que ele possui variáveis categóricas, o que traz uma interpretação diferente para o coeficiente associado a elas.

Vamos observar a dispersão dos dados.

In [ ]:
import matplotlib.pyplot as plt

sns.pairplot(df,
             x_vars= df.drop('mpg', axis=1).columns,
             y_vars='mpg',
             kind='scatter')
plt.show()

Suponha que queremos utilizar apenas a variável *horsepower* para prever o *mpg*. Visualmente, essas variáveis parecem ter uma relação não linear. Vejamos, primeiramente, como fica o modelo de regressão linear.

In [ ]:
# Fit the resgression line using 'OLS'
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt

model = sm.ols(formula='mpg ~ horsepower',
               data=df).fit()
print(model.summary())

# Predição
df['predicted'] = model.predict(df.horsepower)

# Scatterplot com Reta de Regressão

plt.scatter(df.horsepower, df.mpg, color='blue', label='Dados Observados')
plt.plot(df.horsepower, df.predicted, color='red', label='Reta de Regressão')

plt.xlabel('Horsepower')
plt.ylabel('MPG')
plt.title('Scatterplot com Reta de Regressão')
plt.legend()
plt.show()

Note um $R^2 = 0.606$ relativamente baixo, além de um ajuste inadequado que pode ser visualizado na figura acima. De fato, parece haver uma relação não linear entre os dados. Vejamos na sequência um podelo que pode melhors e ajustar a esses dados.

# **Regressão Polinomial**

Seja $x$ a variável independente e $y$ a variável dependente. A função polinomial de grau $p$ que associa ambas as variáveis é dada pela seguinte equação:

$$y = \beta_0 + \beta_1 x + \beta_2 x^2 + \ldots + \beta_p x^p + \epsilon,$$

onde $\beta_0$ é o intercepto e $\beta_1, \ldots, \beta_p$ são os demais parâmetros associados aos dados do modelo.

Note que a equação acima possui uma certa similaridade em relação à regressão linear. Em ambos os modelos, não há não linearidade em relação aos parâmetros. Ou seja, tanto na regressão linear quando no modelo polinomial, em termos do modelo de otimização, ambos são lineares nos parâmetros. Basta trocar $x^2$ por $x_2$, $x^3$ por $x_3$, e assim por diante, para que o modelo fique idêntico ao da regressão linear, porém, com mais parâmetros. Sendo assim, podemos realizar uma transformação das variáveis para se ajustar ao modelo polinomial e, em seguida, realizar o treinamento a partir da regressão linear como vimos na aula passada. Vejamos como fazer isso na sequência.

Vamos criar um modelo de grau 2, ou seja, com a seguinte função:

$$y = \beta_0 + \beta_1 x + \beta_2 x^2 + \epsilon.$$

No Python, a função `PolynomialFeatures` faz a transformação que precisamos, de acordo com o grau do polinômio (`degree`) que desejamos construir.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2,include_bias=False) # Ajuste dos dados para uma função polinomial de grau 2

Note que utilizamos como parâmetro no código acima `include_bias=False`. Isso impõe ao modelo não criar o vetor de dados associado ao $\beta_0$. Iremos deixar essa função para o modelo de regressão linear (como veremos mais para frente). Veja como ficam os dados transformados.

In [ ]:
X_poly = poly.fit_transform(df[['horsepower']])

print(X_poly)

In [ ]:
df[['horsepower2']] = X_poly[:, 1].reshape(-1,1) # Cria uma nova coluna com os dados transformados

df.head()

Temos, agora, uma coluna de dados com os valores originais ($x$) e outra coluna com esses valores ao quadrado ($x^2$). Agora, ao construir o modelo de regressão linear, teremos uma relação não linear entre as variáveis.

In [ ]:
model = sm.ols(formula='mpg ~ horsepower + horsepower2',
               data=df).fit()
print(model.summary())

# Predição
df['predicted'] = model.predict(df[['horsepower','horsepower2']])

# Scatterplot com Reta de Regressão

plt.scatter(df.horsepower, df.mpg, color='blue', label='Dados Observados')
plt.plot(df.horsepower, df.predicted, '.', color='red', label='Reta de Regressão')

plt.xlabel('Horsepower')
plt.ylabel('MPG')
plt.title('Scatterplot com Reta de Regressão')
plt.legend()
plt.show()

Neste novo modelo, encontramos um coeficiente de determinação melhor que o anterior. O que faz sentido, pois no pior dos casos, o novo ajuste desconsideraria o coeficiente associado aos valores ao quadrado e manteria a função linear. No entando, como existe a relação não linear entre as variáveis, o coeficiente associado ao termo quadrático teve um valor expressivo.

Vale ressaltar que usamos apenas uma variável e o grau do polinômio igual a 2. Claramente, podemos incluir mais variáveis no modelo e usar um grau de polinômio maior. Seja $x_1, x_2, \ldots, x_n$ as variáveis independentes e $y$ a variável dependente. A função polinomial de grau $p$ que associa ambas as variáveis é dada pela seguinte equação:

$$y = \beta_0 + \sum_{i=1}^n \beta_i x_i + \sum_{i=1}^n \sum_{j=1}^n \beta_{i,j} x_i x_j + \sum_{i=1}^n \sum_{j=1}^n \sum_{k=1}^n \beta_{i,j,k} x_i x_j x_k + \ldots + \beta_{1,2, \ldots, n} x_1 x_2 \ldots x_n + \epsilon.$$